In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, ViTMAEModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "facebook/vit-mae-base"

image_processor = AutoImageProcessor.from_pretrained(model_name, use_fast=True)

model = ViTMAEModel.from_pretrained(model_name).to(device)


In [3]:
df_allCarabids = pd.read_csv("/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv", sep="\t")
df_insect1M = df_allCarabids[df_allCarabids["filepath"].str.startswith("Insect-1M")].copy()
df_insect1M.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = df_insect1M[cols]
df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']
df = df.dropna()
df.head(2)

/tmp/slurmtmp.818737/ipykernel_2589521/1676424945.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']


,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")

    inputs = image_processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        features = model(**inputs).last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy()

In [5]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"])])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42, stratify=y)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 20278/20278 [06:18<00:00, 53.54it/s] 


In [ ]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
8222,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela willistoni,Ellipsoptera macra,Cicindela repanda,Cicindela duodecimguttata,Cicindela repanda
15309,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela hirticollis,Cicindela hirticollis,Omophron americanum,Cicindela hirticollis,Cicindela hirticollis


In [8]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.094181,0.104139,0.094181,0.080554,0.064796,0.092005
LogisticRegression,LogisticRegression,0.210799,0.212734,0.210799,0.191032,0.118409,0.207635
NearestNeighbor,NearestNeighbor,0.126972,0.183072,0.126972,0.125031,0.082871,0.124722
MLP_Baseline,MLP_Baseline,0.184665,0.181897,0.184665,0.174302,0.111501,0.181975


In [9]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M_dropna/9.ViTMAE-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M_dropna//9.ViTMAE-linear-probing-species-metrics.csv", index=False)